# Manage Hyperparameter Tuning with TensorBoard's HParams 
-----------------------------

Tuning hyperparameters in Machine Learning project could be a real pain. The process is iterative and can take a long time to test all the hyperparameters combination.
But fortunately, HParams, a TensorBoard plugin come to the rescue. It allows to test and find the better combination of hyperparameters.

We start by loading the necessary libraries

In [ ]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np
import datetime

## Load and Prepare MNIST dataset


In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize
x_train = x_train / 255
x_test = x_test/ 255

## Hyperparameters

In [ ]:
HP_ARCHITECTURE_NN = hp.HParam('archi_nn', hp.Discrete(['128,64','256,128']))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.1))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

## Build the model
We will use a simple sequential model on the `MNIST` dataset and we will configure the HParams callback.

In [ ]:
def train_model(hparams, experiment_run_log_dir):
    
    nb_units = list(map(int, hparams[HP_ARCHITECTURE_NN].split(",")))
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(name="FLATTEN"))
    model.add(tf.keras.layers.Dense(units=nb_units[0] , activation="relu", name="D1"))
    model.add(tf.keras.layers.Dropout(hparams[HP_DROPOUT], name="DROP_OUT"))
    model.add(tf.keras.layers.Dense(units=nb_units[1] , activation="relu", name="D2"))
    model.add(tf.keras.layers.Dense(units=10, activation="softmax", name="OUTPUT"))
    
    model.compile(
        optimizer=hparams[HP_OPTIMIZER], 
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=experiment_run_log_dir)
    hparams_callback = hp.KerasCallback(experiment_run_log_dir, hparams)
    
    model.fit(x=x_train, 
              y=y_train, 
              epochs=5,
              validation_data=(x_test, y_test),
              callbacks=[tensorboard_callback, hparams_callback]
             ) 

Next, We will iterate on all the hyperparameters.

In [ ]:
for archi_nn in HP_ARCHITECTURE_NN.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
        for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
            hparams = {
                HP_ARCHITECTURE_NN : archi_nn, 
                HP_OPTIMIZER: optimizer,
                HP_DROPOUT : dropout_rate
            }
            
            experiment_run_log_dir="logs/experiment-" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            
            train_model(hparams, experiment_run_log_dir)

## Start TensorBoard

We then start the Tensorboard application by running the command:

`$ tensorboard --logdir="logs"`

Then we navigate our browser to the folling link:

`http://127.0.0.0:6006`

Note that we can specify a different port if needed by passing for example a `--port 6007` command (for running on port 6007.


We can also start TensorBoard within the notebook through the `%tensorboard --logdir="logs"` command line 

> Note that Tensorboard will be viewable _*as*_ your program is running.

## Visualize the result in the HParams dashboard
We can visualize the results (hyperparameters and metrics) for each run on the table view.<br>
Note that filters and sorts can be applied on the left pane if needed.

Here the screenshot of the table view:

![scalars view](../images/02_tensorboard_hparams_table.png)

On the parallel coordinates view, each axis represents a hyperparameter or a metric and each run is represented by a line. <br>
This visualization allows to identify quickly the better hyperparameters combination.
Here we show how to visualize a vector of parameters with a histogram summary.

![histogram view](../images/02_tensorboard_hparams_coord.png)